# Import

In [129]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval

# Data

## Load

In [130]:
order_products__train = pd.read_csv("../instacart_2017_05_01/order_products__train.csv")
order_products__prior = pd.read_csv("../instacart_2017_05_01/order_products__prior.csv")
products = pd.read_csv("../instacart_2017_05_01/products.csv")
orders = pd.read_csv("../instacart_2017_05_01/orders.csv")

## Train

In [36]:
orders_train = orders[orders['eval_set'] == 'train'].reset_index()
orders_train = orders_train[['user_id', 'order_id']]

In [37]:
order_products__train = order_products__train[['order_id', 'product_id', 'reordered']]
order_products__train = order_products__train.groupby('order_id')['product_id'].apply(list).reset_index()
order_products__train.head()

,order_id,product_id
0,1,"[49302, 11109, 10246, 49683, 43633, 13176, 472..."
1,36,"[39612, 19660, 49235, 43086, 46620, 34497, 486..."
2,38,"[11913, 18159, 4461, 21616, 23622, 32433, 2884..."
3,96,"[20574, 30391, 40706, 25610, 27966, 24489, 39275]"
4,98,"[8859, 19731, 43654, 13176, 4357, 37664, 34065..."


In [38]:
user_products__train = pd.merge(order_products__train, orders_train, on='order_id')
user_products__train = user_products__train[['user_id', 'product_id']]

In [39]:
user_products__train[user_products__train.duplicated(['user_id'], keep=False)]

,user_id,product_id


In [40]:
user_products__train.set_index('user_id', inplace=True)

In [42]:
user_products__train.head()

,product_id
user_id,
112108,"[49302, 11109, 10246, 49683, 43633, 13176, 472..."
79431,"[39612, 19660, 49235, 43086, 46620, 34497, 486..."
42756,"[11913, 18159, 4461, 21616, 23622, 32433, 2884..."
17227,"[20574, 30391, 40706, 25610, 27966, 24489, 39275]"
56463,"[8859, 19731, 43654, 13176, 4357, 37664, 34065..."


### Save/Load dataframes

In [52]:
user_products__train.to_csv("user_products__train.csv")

In [53]:
try:
    user_products__train
except NameError:
    user_products__train = pd.read_csv("user_products__train.csv")
else:
    print("user_products__train already exists")

user_products__train already exists


In [41]:
test_subjects = user_products__train.index

## Prior

In [131]:
orders_prior = orders[orders['eval_set'] == 'prior'].reset_index()
orders_prior = orders_prior[['user_id', 'order_id']]

In [132]:
order_products__prior = order_products__prior[['order_id', 'product_id']]
order_products__prior = order_products__prior.groupby('order_id')['product_id'].apply(list).reset_index()

### Save/Load dataframes

In [87]:
user_products__prior.to_csv('user_products__prior.csv')

In [88]:
try:
    user_products__prior
except NameError:
    user_products__prior = pd.read_csv('user_products__prior.csv')
else:
    print("user_products__prior already exists")

user_products__prior already exists


## Purchase history matrix

In [89]:
cat_columns = ['user_id', 'product_id']
for column in cat_columns:
    user_products__prior[column] = user_products__prior[column].astype('category')

In [124]:
product_purchase_history = coo_matrix((user_products__prior['QTY'], 
                             (user_products__prior['product_id'].cat.codes.copy(), 
                             user_products__prior['user_id'].cat.codes.copy())))
product_purchase_history = purchase_history.tocsr()
user_purchase_history = product_purchase_history.T

### Converters

In [90]:
# Put in id for product and user—get out coorinates for sparse matrix
user_to_coords = {user_id:i for i, user_id in enumerate(user_products__prior['user_id'].cat.categories)}
coords_to_product = {j:product_id for j, product_id in enumerate(user_products__prior['product_id'].cat.categories)}

In [91]:
# Recommener function will output product_ids. Use this dict to then get human names for food
product_id_to_name_dict = pd.Series(products['product_name'].values, index=products['product_id']).to_dict()

# Model